- Time to load 24 hours of data = 1 hr. 
- Time to compute 24 hours of CSD = <5 minutes. 
- Time to validate 24 hours of CSD = 2 hours, 20 min. 
    - Memory footprint tripled during computation. 
- Time to detect 24 hours of sharp waves = 4h
- Time to compute 24 hours of sharp wave properties = 24h
- 24 hour detection is not prohibitively expensive, provided that computation of sharp-wave properties can be parallelized


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt

In [30]:
import numpy as np
import pandas as pd
import seaborn as sns

In [1]:
from ecephys.data import paths
from ecephys.utils.utils import load_df_h5
from ecephys.signal.sharp_wave_ripples import add_states_to_events
import ecephys.plot as eplt

In [18]:
SUBJECT = "Doppio"
HOUR1 = 3600
HOUR24 = HOUR1 * 24

In [49]:
from ecephys.scoring import load_visbrain_hypnogram
from ecephys.utils.utils import load_df_h5

with pd.HDFStore(paths.get_datapath(subject=SUBJECT, condition="BSL-6+18", data="sharp_waves.h5")) as store:
    spws24h, spws24h_meta = load_df_h5(store)

with pd.HDFStore(paths.get_datapath(subject=SUBJECT, condition="REC-24", data="sharp_waves.h5")) as store:
    spws2h, spws2h_meta = load_df_h5(store)
        
hypno2h = load_visbrain_hypnogram(paths.get_datapath(subject=SUBJECT, condition="REC-24", data="hypnogram.txt"))

In [50]:
fig, ax1 = plt.subplots(figsize=(15, 4))
g = sns.histplot(spws24h.start_time, binwidth=10, stat="frequency", ax=ax1)
g.set(xticks=[x * HOUR1 for x in range(24)], xticklabels=[str(x) for x in range(24)], xlabel="Hours from 9AM", ylabel="Density (events per second)", title="SPW Density (10s bins)")
plt.show(g)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [76]:
import seaborn as sns
from ecephys.utils import discard_outliers

isi = discard_outliers(np.diff(spws24.midpoint))
sns.displot(isi, bins=100)

NameError: name 'spws24' is not defined

## Explore gap in 24h detection

In [51]:
HOUR20 = HOUR1 * 20
spws24h[spws24h.start_time < HOUR20].start_time.max()

63883.81895843165

In [52]:
spws24h[spws24h.start_time > HOUR20].start_time.min()

80466.45588293196

In [53]:
80466.45588293196 - 63883.81895843165

16582.636924500308

- Recording starts at 09:08:06 am 3/17
- Large ~5-6Hz artifact begins at 63876.75, but disappears fairly quickly. 
- Channels go fairly flat and look transposed (!?) at 63904 (2:53:10 am 3/18). 14Hz artifact is present. 
- Signal comes back abruptly at 80451.8 (7:25:58 am 3/18). Signal looks okay after that. 
- Total outage time ~16548 seconds (4 hours, 35 minutes, 48 seconds). 


## Compare 24hr and 2hr detection

In [55]:
spws2h = add_event_states(spws2h, hypno2h)

In [56]:
rem_spws2h = spws2h[spws2h['state'] == "REM"].reset_index()
rem_spws2h.index += 1

In [57]:
eplt.lazy_spw_explorer(rem_spws2h, spws2h_meta, SUBJECT, "REC-24")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

- REM events using 2h detection seem legit

In [58]:
start_2h = 3600 * 6
end_2h = 3600 * 8

In [59]:
spws2h_from_24h = spws24h[np.logical_and(spws24h.start_time >= start_2h, spws24h.end_time < end_2h)]

In [60]:
spws2h['start_time'] = spws2h['start_time'] + start_2h
spws2h['end_time'] = spws2h['end_time'] + start_2h
spws2h['midpoint'] = spws2h['midpoint'] + start_2h

In [61]:
spws2h

,start_time,end_time,duration,midpoint,sink_amplitude,sink_integral,state
spw_number,,,,,,,
1,21650.863149,21650.974749,0.1116,21650.918949,-0.030492,-2.236261,Wake
2,21663.575936,21663.619136,0.0432,21663.597536,-0.032680,-0.943868,Wake
3,21689.486710,21689.693110,0.2064,21689.589910,-0.071532,-6.977352,Wake
4,21689.695110,21689.751110,0.0560,21689.723110,-0.037766,-1.532870,Wake
5,21700.349899,21700.429899,0.0800,21700.389899,-0.026501,-1.386314,Wake
...,...,...,...,...,...,...,...
5011,28795.503964,28795.541964,0.0380,28795.522964,-0.027557,-0.753133,N2
5012,28795.847564,28795.893564,0.0460,28795.870564,-0.037423,-1.061557,N2
5013,28797.128363,28797.187163,0.0588,28797.157763,-0.046602,-1.558581,N2


In [62]:
hypno2h['start_time'] = hypno2h['start_time'] + start_2h
hypno2h['end_time'] = hypno2h['end_time'] + start_2h

In [63]:
hypno2h

,state,end_time,start_time,duration
0,Wake,21773.990005,21600.000000,173.990005
1,N1,21779.990005,21773.990005,6.000000
2,Wake,21797.990005,21779.990005,18.000000
3,N1,21803.990005,21797.990005,6.000000
4,N2,21815.990005,21803.990005,12.000000
...,...,...,...,...
197,REM,28565.990234,28409.990234,156.000000
198,N1,28601.990234,28565.990234,36.000000
199,Wake,28619.990234,28601.990234,18.000000
200,N1,28625.990234,28619.990234,6.000000


In [64]:
spws2h_from_24h = add_event_states(spws2h_from_24h.copy(), hypno2h)

In [65]:
rem_spws2h_from_24h = spws2h_from_24h[spws2h_from_24h['state'] == "REM"].reset_index()
rem_spws2h_from_24h.index += 1

In [67]:
eplt.lazy_spw_explorer(rem_spws2h_from_24h, spws24h_meta, SUBJECT, "BSL-6+18")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

- Anecdotally, it seems that REM events using 24hr detection may include more false positives than using 2hr detection. 

In [70]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 4))
nbins = int(7200 / 10)
ax1.hist(spws2h.start_time, bins=nbins)
ax1.set_title("SPW density when detection threshold is set using 2h of data")
ax1.set_ylabel("SPW count (10s bins)")
eplt.plot_hypnogram_overlay(hypno2h, ax=ax1)
ax2.set_title("SPW density when detection threshold is set using 24h of data")
ax2.hist(spws2h_from_24h.start_time, bins=nbins)
ax2.set_ylabel("SPW count (10s bins)")
eplt.plot_hypnogram_overlay(hypno2h, ax=ax2)
ax2.set_xlabel("Time (seconds from 9am)")
plt.subplots_adjust(hspace=0.5)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [69]:
(len(spws2h_from_24h) - len(spws2h)) / len(spws2h)

0.2839481555333998

- Observation: Nearly 30% more events detected using 24hrs. 
- Observation: The increase in the number of detected events appears most noticeable during REM
- The aforementioned difference may be due to the ~4.5 hour artifact period, which will result in the mean SR sink being pulled artificially low. 
- Observation: Despite the aforementioned differences, density profile appears nearly identical
- 24hr detection may warrant a lower threshold than 2hr detection
- Wake events are likely to be highly reliable using 24hr detection
- Obtaining a detection threshold during a restricted interval (e.g. 2 hours of circadian-matched baseline) and applying it everywhere may be most principled, practical (esp. in the case of missing or artificatual data) approach short of doing 48-hour detection

In [71]:
print(f"2h CSD: lambda={spws2h_meta['lambd']}, R={spws2h_meta['R']}")
print(f"24h CSD: lambda={spws24h_meta['lambd']}, R={spws24h_meta['R']}")

2h CSD: lambda=0.0003822395851068327, R=0.23
24h CSD: lambda=0.0039366356376800745, R=0.23


In [72]:
def percent_difference(x, y):
    assert x > 0
    assert y > 0
    return (np.abs(x - y) / ((x + y) / 2)) * 100

In [73]:
percent_difference(spws24h_meta['lambd'], spws2h_meta['lambd'])

164.5982284378034

In [75]:
48000/(24-4.5)*24

59076.92307692308

- kCSD searches through: min lambda = 1e-12, max lambda 0.0126
- 2h CSD: lambda=0.0003822395851068327, R=0.23
- 24h CSD: lambda=0.0039366356376800745, R=0.23
- Probably safe to use 2h values everywhere 
    - Using the same values (e.g. those obtained from the first two hours of recovery sleep) everywhere may be more principled

## Archive: Detect bout-by-bout [Doesn't work]

In [62]:
from ecephys.scoring import load_visbrain_hypnogram

hypnogram = load_visbrain_hypnogram(paths.get_datapath(subject=SUBJECT, condition=CONDITION, data="hypnogram.txt"))

In [66]:
fig, ax1 = plt.subplots(figsize=(24, 3))
plt.hist(spws.start_time, bins=int(7200/10))
eplt.plot_hypnogram_overlay(hypnogram, ax=ax1)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [90]:
def get_bout_spws(time, sr_csd, bout):
    bout_mask = np.logical_and(time >= bout['start_time'], time <= bout['end_time'])
    bout_spws = detect_sharp_waves(time[bout_mask], sr_csd[:, bout_mask])
    bout_spws['state'] = bout['state']
    return bout_spws

def get_spws_bout_by_bout(time, sr_csd, hypnogram):
    spws = pd.DataFrame(columns=["spw_number", "start_time", "end_time"])
    spws.set_index('spw_number', inplace = True)
    for index, row in hypnogram.iterrows(): 
        bout_spws = get_bout_spws(time, sr_csd, row)
        spws = spws.append(bout_spws)
        
    return spws
        

In [105]:
bspws = get_spws_bout_by_bout(time, sr_csd, hypnogram)
bspws["duration"] = get_durations(bspws)
bspws["midpoint"] = get_midpoints(bspws)

In [106]:
fig, ax1 = plt.subplots(figsize=(24, 3))
plt.hist(bspws.start_time, bins=int(7200/10))
eplt.plot_hypnogram_overlay(hypnogram, ax=ax1)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [110]:
rem_spws = bspws[bspws['state'] == "REM"].reset_index()

In [115]:
from pathlib import Path

metadata = dict(
    csd_chans=hpc_chans,
    detection_chans=sr_chans,
    intersite_distance=intersite_distance,
    gdx=interestimate_distance,
    lambd = k.lambd,
    R = k.R,
    detect_states=["Wake", "N1", "N2", "REM"],
    detection_zscore_threshold=2.5,
    boundary_zscore_threshold=1,
    minimum_duration=0.005,
)
rem_spws_path = Path('/Volumes/neuropixel/Data/CNPIX4-Doppio/3-18-2020_g0_t3.imec0.rem_spws.h5')
store_df_h5(rem_spws_path, rem_spws, **metadata)

In [116]:
eplt.lazy_spw_explorer(rem_spws, metadata, SUBJECT, CONDITION)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()